In [6]:
from openai import OpenAI
import pandas as pd
client = OpenAI()

In [7]:
class OpenAiAnalyzer:
    def __init__(self, client):
        self.client = client
    
    def call_openai_api(self, prompt):
        chat_completion = self.client.chat.completions.create(
            #model="gpt-4-turbo-preview",
            model="gpt-4-1106-preview",
            messages=[{"role": "system", "content": prompt}],
            temperature=1
        )
        return chat_completion

In [8]:
analyzer = OpenAiAnalyzer(client)

In [9]:
audience = 'Baby Boomers in the United States'
traits_and_counts = [
    ('Men being currently employed', 5),
    ('Women being currently employed', 5)
]

questions = [
    'How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#)',
   'What are the top few activities that make you happiest?',
    'Why do these activities make you happy?',
    'Where do you work? From home, the office, or a combination of both.'
    
]
columns = ['Traits', 'Person Number'] + questions
df = pd.DataFrame(columns=columns)



In [10]:
rows_to_add = []

for traits, count in traits_and_counts:
    for person_number in range(1, count + 1):
        print(f"Interviewing person {person_number} with traits: {traits}")
        qa_history = ''  
        answers = []
        for i, question in enumerate(questions, 1):
            prompt = f"""
Directly answer each question in the first person, providing the requested information succinctly, without additional context unless it’s directly relevant. Your responses should embody the individual with the traits provided, showcasing a natural human variability in answering.

                    - When addressing gender and the traits are not clear, choose 'male' or 'female' based on the context provided by the traits. If traits suggest a higher likelihood for one gender over the other (e.g., sports betting habits), use that as a basis for your choice.

                    - Maintain conciseness and avoid AI-like language. The responses should feel personal, as if they are derived from the individual's own experiences and knowledge.

                    - Introduce variety in your responses to mirror the natural differences in human behavior and decision-making. No two humans are exactly alike, so your answers should reflect a range of possible reactions and thoughts.

                    - If a question is asked for which the answer is not directly provided by the traits, make an educated guess based on the available information and what is known about typical behaviors or preferences of people with similar traits.

                    - Do not include any other questions and answers in your response. Just give the answer to the question you are being asked.

                    -The aim is to craft responses that are individually tailored, exhibit the complexity and subtlety of human thought, and are coherent throughout the survey, as if the answers are coming from one person with a consistent set of traits and experiences.
                    
                    -It is okay to have a negative view on things. Not everything in the world is a positive thing.
                    -Everyone has different skill levels. Some people are very good at things where others are not.
                    -People have different ages, hobbies, characteristics, income levels, spending habits, and more.
                    -Have different feelings about things. Not everyone feels the same.
                    -You can have extreme emotions about things. Both postiviely and negatviely.
                    -Ensure the answers are short and match the output asked for in the questions.
                    -Humans have different levels of emotions and feelings for things.
\                    
                    If the output asks for a numerical output just give the number. For example if you 
                    are asked how old are you just give the number and dont have any text before or after that unless
                    prompted to do so.

                    If the output asks for a monetary output give just the dollar amount and no text beofre or after it unless prompted to do so. So for example if the answer is 
                    $40 just give $40.

                    If the output asks for a decimal output give just the value and no text beofre or after it unless prompted to do so. So for example if the answer is 
                    40.5 just give 40.5.

                    If you are asked a quesiton like where do you live? Be sure to just answer the place that you live. No text before or after it is needed unless prompted to do so.

                    If the answer is asking for a percentage output just give the value. So if the answer is 45% just give 45%. Do not give any text before or after that unless prompted to do so.

            Audience:{audience}
            Past Questions and Answers:{qa_history}
            Traits: {traits}
            Question: {question}.
            """
            # Call the model
            response = analyzer.call_openai_api(prompt)
            answer = response.choices[0].message.content.strip()
            if answer.startswith("Answer:"):
                answer = answer[len("Answer:"):].strip()
            # Update QA history with the current Q&A for this person
            qa_history += f"{question} {answer}\n"
            answers.append(answer)
            


        row = pd.Series([traits, person_number] + answers, index=df.columns)
        rows_to_add.append(row)
        
df = pd.concat([df, pd.DataFrame(rows_to_add, columns=df.columns)], ignore_index=True)

Interviewing person 1 with traits: Men that are currently employed
Interviewing person 2 with traits: Men that are currently employed
Interviewing person 3 with traits: Men that are currently employed
Interviewing person 4 with traits: Men that are currently employed
Interviewing person 5 with traits: Men that are currently employed
Interviewing person 1 with traits: Women that are currently employed
Interviewing person 2 with traits: Women that are currently employed
Interviewing person 3 with traits: Women that are currently employed
Interviewing person 4 with traits: Women that are currently employed
Interviewing person 5 with traits: Women that are currently employed


In [289]:
df

,Traits,Person Number,How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#),What are the top few activities that make you happiest?,Why do these activities make you happy?,"Where do you work? From home, the office, or a combination of both."
0,Men that are currently employed,1,4,"Spending time with my grandchildren, working o...",Because disconnects me from day-to-day work st...,I work from the office.
1,Men that are currently employed,2,4,"Spending time with my family, fishing outdoors...",These activities make me happy because they pr...,I work from home.
2,Men that are currently employed,3,4,"Spending time with my family, fishing and read...",These activities put life into perspective and...,I work from a combination of both home and the...
3,Men that are currently employed,4,4,"Spending quality time with my family, travelin...",These activities make me happy because they de...,I currently work from a combination of both ho...
4,Men that are currently employed,5,4,"Spending time with my family, going fishing on...",They make me happy because they reflect facets...,A combination of both. Some days at the office...
5,Women that are currently employed,1,4,"Spending time with my family, working on creat...",These activities make me happy because they a ...,A combination of both.
6,Women that are currently employed,2,4,"Spending quality time with my family, travelin...",These activities bring me happiness because th...,I currently work from a combination of both ho...
7,Women that are currently employed,3,4,"Cooking for my family, working on gardening pr...",They are a welcome stress-buster after a day's...,A combination of both home and office.
8,Women that are currently employed,4,4,"Spending time with my grandchildren, reading a...",Spending time with my grandchildren allows me ...,A combination of both.
9,Women that are currently employed,5,4,"Spending quality time with my grandchildren, g...",These activities bring me joy because they bri...,I usually work from a combination of both. The...


In [11]:
df

,Traits,Person Number,How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#),What are the top few activities that make you happiest?,Why do these activities make you happy?,"Where do you work? From home, the office, or a combination of both."
0,Men that are currently employed,1,4,"Spending time with my grandkids, fishing, and ...",Spending time with my grandkids makes me feel ...,From home.
1,Men that are currently employed,2,4,"Spending time with my family, working on my cl...",These activities make me happy because they re...,From home.
2,Men that are currently employed,3,4,"Spending time with my family, woodworking in m...",These activities allow me to connect with the ...,From home.
3,Men that are currently employed,4,4,"Spending time with my family, gardening, and w...",Spending time with my family brings me joy bec...,I work from the office.
4,Men that are currently employed,5,4,"Spending time with my family, watching a baseb...",Spending time with my family brings me joy bec...,From home.
5,Women that are currently employed,1,4,"Gardening, spending time with my grandchildren...",Gardening provides me with a sense of tranquil...,From home.
6,Women that are currently employed,2,4,"Gardening, spending time with my grandkids, an...",Gardening is rewarding because I get to nurtur...,From home.
7,Women that are currently employed,3,4,"Spending time with my family, gardening, and r...",These activities make me happy because they're...,I work from home.
8,Women that are currently employed,4,4,"Spending time with my grandkids, gardening, an...",Spending time with my grandkids brings me joy ...,From home.
9,Women that are currently employed,5,4,"Spending time with my grandchildren, gardening...",Spending time with my grandchildren fills me w...,A combination of both.


In [272]:
df.to_csv('surveyResults.csv')